In [1]:
from azureml.core.authentication import AzureCliAuthentication
from azureml.core.workspace import Workspace
import pandas as pd

cli_auth = AzureCliAuthentication()
ws = Workspace.from_config(auth=cli_auth)
ws.get_details()

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.2.0
Subscription ID,b9f1c816-0637-419c-b0f6-a8392690c7aa
Workspace,paydmlws91
Resource Group,paydmlrg91
Location,westeurope


### Load the Data - local
We can load the payments dataset we want to score from csv files locally. The data as been processed to only contains features needed to do the scoring. This appoach allows the data to be in different files which are all collected into one dataset and then scored.

In [2]:
import glob
import pandas as pd

#path = r'C:\DRO\DCL_rawdata_files' # use your path
all_files = glob.glob("./data/ml_scoring_input/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    print(f"Loading.. {filename}")

df = pd.concat(li, axis=0, ignore_index=True)


# preview the first 3 rows of the dataset
print(f"Loaded {len(df.index)} rows.")

df.head(3)

Loading.. ./data/ml_scoring_input\UCI_Credit_Card_INPUT_1.csv
Loading.. ./data/ml_scoring_input\UCI_Credit_Card_INPUT_2.csv
Loading.. ./data/ml_scoring_input\UCI_Credit_Card_INPUT_3.csv
Loaded 25 rows.


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,260000.0,2,1,2,51,-1,-1,-1,-1,-1,...,9966,8517,22287,13668,21818,9966,8583,22301,0,3640
1,630000.0,2,2,2,41,-1,0,-1,-1,-1,...,6500,6500,6500,2870,1000,6500,6500,6500,2870,0
2,250000.0,1,1,2,29,0,0,0,0,0,...,63561,59696,56875,55512,3000,3000,3000,3000,3000,3000


In [3]:
from azureml.core.model import Model
import json

keyvault = ws.get_default_keyvault()
model_info = json.loads(keyvault.get_secret(name="PAYMENTSMODEL"))

model_name = model_info['NAME']
model = Model(ws, model_name)
model

Model(workspace=Workspace.create(name='paydmlws91', subscription_id='b9f1c816-0637-419c-b0f6-a8392690c7aa', resource_group='paydmlrg91'), name=AutoMLc50723a2133, id=AutoMLc50723a2133:2, version=2, tags={'area': 'risk', 'type': 'classification'}, properties={'Code': 'https://github.com/riedwaanb/DefaultPayments'})

In [4]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

environment = Environment("LocalDeploy")
environment.python.conda_dependencies = CondaDependencies("inference/env.yml")

inference_config = InferenceConfig(entry_script="inference/score.py",
                                   environment=environment)

### Deploy Model as a Local Docker Web Service
Make sure you have Docker installed and running.

Note that the service creation can take quite a few minutes... be patient. 

NOTE:
The Docker image runs as a Linux container. If you are running Docker for Windows, you need to ensure the Linux Engine is running:

#### PowerShell command to switch to Linux engine
& 'C:\Program Files\Docker\Docker\DockerCli.exe' -SwitchLinuxEngine

In [5]:
from azureml.core.webservice import LocalWebservice

# This is optional but recommended, if not provided Docker will choose a random unused port
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "default-payments", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry paydmlws918ef0ad0f.azurecr.io
Logging into Docker registry paydmlws918ef0ad0f.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM paydmlws918ef0ad0f.azurecr.io/azureml/azureml_48a4ac85e4690e5ca62e99f1e0611fb1
 ---> 54ac75b7aeb1
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 0d878b1891ac
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImI5ZjFjODE2LTA2MzctNDE5Yy1iMGY2LWE4MzkyNjkwYzdhYSIsInJlc291cmNlR3JvdXBOYW1lIjoicGF5ZG1scmc5MSIsImFjY291bnROYW1lIjoicGF5ZG1sd3M5MSIsIndvcmtzcGFjZUlkIjoiZjYwOGQ3NzMtMjhkNy00NzdhLWE2Y2QtMWFkMjliMTJlNTY0In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in f8e61b928305
 ---> aa5c86b5b80c
Step 4/5 : RUN mv '/var/azureml-app/tmprfpsyw47.py' /var/azureml-app/main.py
 ---> Running in 221b9d4fd474
 ---> a15b16cf91b1
Step 5/5 : CMD ["runsvdir

#### Retrieve the Webservice Port
If you need to call the webservice without the AzureML SDK, use this following port.

In [6]:
print('Local service port: {}'.format(local_service.port))

Local service port: 6789


### Score the data
Loop through the dataframe scoring chunks of data at a time

In [7]:
chunck_size = 10
scores = []

# Calculate and return the next chunck
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

# Loop through all the Chuncks
for i in chunker(df, chunck_size):
    input_data = i.to_json(orient='records')              # Convert to json using rows
    input_data = f'{{"data" : {input_data}}}'             # The webservice is expecting this format
    print(f"Scoring:\n{input_data}")
    score_input = bytes(input_data, encoding='utf-8')     # Ensure its encoded correctly
    resp = local_service.run(input_data=score_input)      # Call the webservice
    score_output = json.loads(resp).get("result")         # retrieve the score from the format: {'result': [0]}
    print(f"Recieved:\n{score_output}\n----------------------------")
    scores.extend(score_output)                           # Built a scores array

df['score'] = scores                                      # Create the score column with all the scores
print(f"All Scores:\n{scores}")

Scoring:
{"data" : [{"LIMIT_BAL":260000.0,"SEX":2,"EDUCATION":1,"MARRIAGE":2,"AGE":51,"PAY_0":-1,"PAY_2":-1,"PAY_3":-1,"PAY_4":-1,"PAY_5":-1,"PAY_6":2,"BILL_AMT1":12261,"BILL_AMT2":21670,"BILL_AMT3":9966,"BILL_AMT4":8517,"BILL_AMT5":22287,"BILL_AMT6":13668,"PAY_AMT1":21818,"PAY_AMT2":9966,"PAY_AMT3":8583,"PAY_AMT4":22301,"PAY_AMT5":0,"PAY_AMT6":3640},{"LIMIT_BAL":630000.0,"SEX":2,"EDUCATION":2,"MARRIAGE":2,"AGE":41,"PAY_0":-1,"PAY_2":0,"PAY_3":-1,"PAY_4":-1,"PAY_5":-1,"PAY_6":-1,"BILL_AMT1":12137,"BILL_AMT2":6500,"BILL_AMT3":6500,"BILL_AMT4":6500,"BILL_AMT5":6500,"BILL_AMT6":2870,"PAY_AMT1":1000,"PAY_AMT2":6500,"PAY_AMT3":6500,"PAY_AMT4":6500,"PAY_AMT5":2870,"PAY_AMT6":0},{"LIMIT_BAL":250000.0,"SEX":1,"EDUCATION":1,"MARRIAGE":2,"AGE":29,"PAY_0":0,"PAY_2":0,"PAY_3":0,"PAY_4":0,"PAY_5":0,"PAY_6":0,"BILL_AMT1":70887,"BILL_AMT2":67060,"BILL_AMT3":63561,"BILL_AMT4":59696,"BILL_AMT5":56875,"BILL_AMT6":55512,"PAY_AMT1":3000,"PAY_AMT2":3000,"PAY_AMT3":3000,"PAY_AMT4":3000,"PAY_AMT5":3000,"PAY_

Recieved:
[0, 1, 0, 0, 1]
----------------------------
All Scores:
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1]


In [8]:
df.head(3)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,score
0,260000.0,2,1,2,51,-1,-1,-1,-1,-1,...,8517,22287,13668,21818,9966,8583,22301,0,3640,0
1,630000.0,2,2,2,41,-1,0,-1,-1,-1,...,6500,6500,2870,1000,6500,6500,6500,2870,0,0
2,250000.0,1,1,2,29,0,0,0,0,0,...,59696,56875,55512,3000,3000,3000,3000,3000,3000,0


### Filter to see only payments with score = 1

In [9]:
scored = df.loc[df['score'] == 1]
scored

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,score
5,70000.0,2,2,2,26,2,0,0,2,2,...,44006,46905,46012,2007,3582,0,3601,0,1820,1
18,210000.0,1,2,1,34,3,2,2,2,2,...,2500,2500,2500,0,0,0,0,0,0,1
21,80000.0,1,2,2,34,2,2,2,2,2,...,77519,82607,81158,7000,3500,0,7000,0,4000,1
24,30000.0,1,2,2,37,4,3,2,-1,0,...,20878,20582,19357,0,0,22000,4200,2000,3100,1


### Save Scored Data Locally

In [10]:
import time
filename = time.strftime("./data/ml_scoring_output/scored-%Y%m%d-%H%M%S.csv")
scored.to_csv(filename)